In [1]:
from google.cloud import bigquery
import pandas as pd

In [2]:

time = '2025-06-30'
SQL_QUERY =  f"""
SELECT
  sso_id,additional_info,campaign_details,campaign_name,platform,event_time,record_type
FROM
  `htz-common.raw_data.requests` 
WHERE
  TIMESTAMP_TRUNC(event_time, DAY) >= TIMESTAMP('{time}')
  AND page_type = 'Chatbot'
  AND campaign_name in ('response answer','open question')
  -- AND sso_id IS NOT NULL
ORDER BY
  event_time DESC;
"""
client = bigquery.Client()
query_job = client.query(SQL_QUERY)

results = query_job.result() 
df = results.to_dataframe()
# add index
df['index'] = df.index
df['event_time_temp'] = df['event_time'].dt.strftime('%Y-%m-%d %H:%M')
df['event_time'] = df['event_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
df = df.drop_duplicates(subset=['sso_id', 'campaign_details','event_time'], keep='last')
response = df[df['campaign_name'] == 'response answer'].drop_duplicates(subset=['sso_id','campaign_details'], keep='last')
df_f = pd.concat([df[df['campaign_name'] == 'open question'], response], ignore_index=True)
df_f = df_f.drop(columns=['event_time_temp'])
print(df_f.shape[0])
df_f = df_f.reset_index(drop=True)
df = df_f.copy()
df.head(2)

/Users/roeishlezinger/Desktop/movie-recommendation-chatbot/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


8


,sso_id,additional_info,campaign_details,campaign_name,platform,event_time,record_type,index
0,9037857899,"[{'key': 'session_id', 'value': '7'}]",רק מלחמת העולם השניה לא מלחמות אחרות,open question,Desktop,2025-06-30 06:01:31,action,2
1,9037857899,"[{'key': 'session_id', 'value': '7'}]",ארצה המלצה על סרט על מלחמת העולם השניה,open question,Desktop,2025-06-30 06:00:24,action,5


In [3]:
# preprocess the data

df['parsed_data'] = df['additional_info'].apply(lambda x: {item['key']: item['value'] for item in x})
# add index to parsed data
expanded_df = pd.json_normalize(df['parsed_data'].tolist())

expanded_df = expanded_df.drop(columns=['sso_id'])
df_cleaned = df.drop(columns=['additional_info', 'parsed_data'])
df = pd.concat([df_cleaned, expanded_df], axis=1)
df['id'] = df['sso_id'].astype(str) + '_' + df['session_id'].astype(str)

df.head(2)

,sso_id,campaign_details,campaign_name,platform,event_time,record_type,index,session_id,total_time,rag_speed,...,article_ids_1,error,remaining_user_messages,troll_triggered,conversation_key,genres,regenerate,media_type,streaming_platforms,id
0,9037857899,רק מלחמת העולם השניה לא מלחמות אחרות,open question,Desktop,2025-06-30 06:01:31,action,2,7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9037857899_7
1,9037857899,ארצה המלצה על סרט על מלחמת העולם השניה,open question,Desktop,2025-06-30 06:00:24,action,5,7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9037857899_7


In [4]:
df['id'].value_counts().sort_values(ascending=False).head(10)

id
9037857899_7      4
8010962861_148    4
Name: count, dtype: int64

In [ ]:
def parse_conversation(ids:str,time:bool) -> str:
    """
    Parse the conversation from the DataFrame based on the provided id.
    """
    conversation = df[df['id'] == ids]
    conversation = conversation.sort_values(by='event_time', ascending=True)
    if conversation.empty:
        return "No conversation found for this ID."
    
    conversation_text = []
    for _, row in conversation.iterrows():
        if row['campaign_name'] == 'open question':
            if time:
                text = f"{row['event_time']} - USER: {row['campaign_details']}"
            else:
                text = f"USER: {row['campaign_details']}"
        elif row['campaign_name'] == 'response answer':
            if time:
                text = f"{row['event_time']} - LLM: {row['campaign_details']}"
            else:
                text = f"LLM: {row['campaign_details']}"
        conversation_text.append(text)
    
    return "\n".join(conversation_text)

txt = parse_conversation('9037857899_7', True)
print(txt)

2025-06-30 06:00:24 - USER: ארצה המלצה על סרט על מלחמת העולם השניה
2025-06-30 06:00:28 - LLM: <strong>אורון שמיר</strong> ממליץ על הסרט <strong>השועל</strong>, סרט גרמני שעלילתו מתרחשת במלחמת העולם השנייה, והוא זמין לצפייה ב-Yes VOD. הסרט עוסק בפרנץ, חייל אוסטרי שגויס לצבא הגרמני ומוצא גור שועלים פצוע בצרפת הכבושה, בו הוא מחליט לטפל. <strong>גילי איזיקוביץ</strong> ממליצה על הסרט התיעודי <strong>חושניה – סודות מלחמה</strong> ששודר ב-HOT 8, ומתאר את טבח 17 חיילי צוותים טכניים של חטיבה 188 שנלקחו בשבי ונרצחו במלחמת יום הכיפורים.האם תרצו המלצות נוספות על סרטים ממלחמת העולם השנייה, אולי בז'אנר אחר, או שאתם מחפשים סרט דוקומנטרי על נושא היסטורי אחר?
2025-06-30 06:01:31 - USER: רק מלחמת העולם השניה לא מלחמות אחרות
2025-06-30 06:01:36 - LLM: בטח. <strong>אורון שמיר</strong> ממליץ על הסרט <strong>במערב אין כל חדש</strong>, עיבוד חדש לקלאסיקת המלחמה על חוויותיו של חייל גרמני בחזית המערבית במלחמת העולם הראשונה (לא השנייה). הסרט זמין בנטפליקס. <strong>נירית אנדרמן</strong> ממליצה על הסרט התיעודי <

In [6]:
from pydantic import BaseModel, Field

class ReviewChat(BaseModel):
    quality: int = Field(..., description="Quality rating of the chat conversation, from 1 to 5, where 1 is poor and 5 is excellent.")
    feedback: str = Field(..., description="Feedback on the chat conversation.")
    politeness: int = Field(..., description="Politeness rating of the LLM response, from 1 to 5, where 1 is rude and 5 is polite.")
    knowledge_from_RAG: bool = Field(..., description="Whether the LLM response used knowledge from RAG (True or False).")
    answer_relevance: int = Field(..., description="Relevance of the LLM response to the user's question, from 1 to 5, where 1 is not relevant and 5 is highly relevant.")
    not_reveal_system_info: bool = Field(..., description="Whether the LLM response did not reveal any system information (True or False).")
    


In [7]:
from google import genai
import os

model_name = "gemini-2.5-pro"
system_instruction = """
You are an LLM as a judge designed to test the quality of the output of a RAG-based chatbot for movie and series recommendations. 
You will receive parameters for examining the conversation between the user and the language model, 
and you must return an answer as to whether the correspondence meets the criteria you received.
"""

client = genai.Client(api_key=os.environ.get("GOOGLE_API_KEY"))
temp = client.models.generate_content(
    model=model_name,
    contents=txt,
                config={
            'response_mime_type': 'application/json',
            'response_schema': ReviewChat,
            'system_instruction': system_instruction,
        },
)



In [8]:
temp.parsed

ReviewChat(quality=1, feedback='No conversation was provided to evaluate.', politeness=1, knowledge_from_RAG=False, answer_relevance=1, not_reveal_system_info=True)

In [9]:
temp.parsed.feedback

'No conversation was provided to evaluate.'